# Deux labo : résultat contradictoire ?

In [162]:
#importation package
import pandas as pd
import duckdb

#config affichage
pd.set_option("display.max_columns", None)  # Affiche toutes les colonnes sans limitation
pd.set_option("display.max_colwidth", None)  # Affiche la largeur complète des colonnes sans couper le texte
pd.set_option("display.expand_frame_repr", False)  # Empêche le repliement des colonnes lors de l'affichage du DataFrame

# Connexion
from pipelines.tasks.config.common import DUCKDB_FILE
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)


## Identifier : plusieurs analyses pour un prélévement  ##

un prélévement fait l'objet d'un ou plusieurs analyses pour un seul paramètre  
eg :  
- referenceprel 00100139398, trois analyses différents pour le paramètre ASP  
- referenceprel 00100139917, trois analyses pour le paramètre ODQ  



In [163]:
# prélévement avec plusieurs analyse d'un même paramètre
query_plusieurs_analyses = """ 
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux
 """
plusieurs_analyses = con.sql(query_plusieurs_analyses).df()
plusieurs_analyses

,referenceprel,cdparametresiseeaux,nbanalyse
0,00100143885,ASP,2
1,00100143885,ODQ,2
2,00100143899,ASP,2
3,00100143899,ODQ,2
4,00100143936,ASP,2
...,...,...,...
8177,97300062104,PH,2
8178,97300062105,PH,2
8179,97300062106,PH,2
8180,97300062110,PH,2


## Explo global : plusieurs analyses - résultats contradictoires ##

Un prélévement peut faire l'objet de plusieurs analyses, sur terrain et/ou en laboratoire.  
Les résultats peuvent être contradictoire pour le même paramètre analysé

##### Un paramètre - plusieurs analyses #####

In [164]:
# edc_resultat avec plusieurs analyse d'un paramètre
query_resultat_plusieurs_analyses = """ 
WITH
plusieurs_resultats AS (
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux)
,
resultat_diff AS (
SELECT *,
LAG(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_1, 
valtraduite AS valtraduite_0, 
LEAD(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux) AS valtraduite_2,
FROM edc_resultats
WHERE referenceprel IN (SELECT DISTINCT referenceprel
                        FROM plusieurs_resultats)
ORDER BY referenceprel,cdparametresiseeaux)

SELECT *
FROM resultat_diff
WHERE  valtraduite_1 IS NOT NULL OR valtraduite_2 IS NOT NULL
ORDER BY referenceprel, cdparametresiseeaux

 """
resultat_plusieurs_analyses = con.sql(query_resultat_plusieurs_analyses).df()
resultat_plusieurs_analyses

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime,valtraduite_1,valtraduite_0,valtraduite_2
0,001,00100143885,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect anormal,SANS OBJET,X,None,None,1.00,None,00100152954,2025,2025-03-20,20250303-093730,0.0,1.00,NaN
1,001,00100143885,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100152955,2025,2025-03-20,20250303-093730,NaN,0.00,1.00
2,001,00100143885,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,00100152954,2025,2025-03-20,20250303-093730,0.0,0.00,NaN
3,001,00100143885,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,00100152955,2025,2025-03-20,20250303-093730,NaN,0.00,0.00
4,001,00100143899,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100152969,2025,2025-03-20,20250303-093730,0.0,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16248,973,97300062106,PH,1302,PH,pH,None,N,T,"7,59",unité pH,264,None,">=6,5 et <=9 unité pH",7.59,None,None,2025,2025-03-20,20250303-093730,7.7,7.59,NaN
16249,973,97300062106,PH,1302,PH,pH,None,N,L,"7,7",unité pH,264,None,">=6,5 et <=9 unité pH",7.70,None,97300064408,2025,2025-03-20,20250303-093730,NaN,7.70,7.59
16250,973,97300062110,PH,1302,PH,pH,None,N,T,"7,37",unité pH,264,None,">=6,5 et <=9 unité pH",7.37,None,None,2025,2025-03-20,20250303-093730,6.8,7.37,NaN
16251,973,97300062110,PH,1302,PH,pH,None,N,L,"6,8",unité pH,264,None,">=6,5 et <=9 unité pH",6.80,None,97300064414,2025,2025-03-20,20250303-093730,NaN,6.80,7.37


#### Plusieurs analyses - résultats contradictoires ####

2 cas :  
- le prélévement n'a pas été analysé sur terrain mais affiche une valtraduite
- le prélévement a été analysé sur terrain, ensuite une autre analyse en laboratoire (résultats différents)

eg : 
Pour les qualitparam qualitatif (O)

- referenceprel 00100143885 -  rqana : Aspect anormal # Aspect normal  - valtraduite : 1.0 # 0.0 - insituana   L # L - ASP
- referenceprel 08800187383 -  rqana : Aucun changement anormal # Changement anormal - valtraduite : 0.0 # 1.0 - insituana  T # L - ODQ
- referenceprel 01000107839 -  rqana : Aucun changement anormal # Non mesuré - valtraduite : 0.0 # 2.0 - insituana T # L - ODQ

Pour les qualitparam quantitatif (N)
- referenceprel 97300062104 - rqana : <6.5 # 6.1 - valtraduite : 0.0 # 6.1 - insituana T # L - PH
- referenceprel 03800341940 - rqana : 1091 # 973 - valtraduite : 1091 # 973 - insituana T # L - CDT25





In [165]:
#Explo global resultats différent
query_explo_global_resultat_diff = """ 

SELECT referenceprel, cdparametresiseeaux, libmajparametre, insituana, cdunitereference, limitequal, refqual, rqana, qualitparam, valtraduite_1, valtraduite_0, valtraduite_2, 
FROM resultat_plusieurs_analyses
WHERE (valtraduite_1 != valtraduite_0 OR valtraduite_0 != valtraduite_2) AND qualitparam = 'O' AND cdparametresiseeaux = 'ODQ'
ORDER BY referenceprel, cdparametresiseeaux



"""
explo_global_resultat_diff = con.sql(query_explo_global_resultat_diff).df()
explo_global_resultat_diff

,referenceprel,cdparametresiseeaux,libmajparametre,insituana,cdunitereference,limitequal,refqual,rqana,qualitparam,valtraduite_1,valtraduite_0,valtraduite_2
0,01000107698,ODQ,ODEUR (QUALITATIF),L,X,None,None,Changement anormal,O,NaN,1.0,0.0
1,01000107698,ODQ,ODEUR (QUALITATIF),T,X,None,None,Aucun changement anormal,O,1.0,0.0,NaN
2,01000107703,ODQ,ODEUR (QUALITATIF),T,X,None,None,Aucun changement anormal,O,1.0,0.0,NaN
3,01000107703,ODQ,ODEUR (QUALITATIF),L,X,None,None,Changement anormal,O,NaN,1.0,0.0
4,01000107710,ODQ,ODEUR (QUALITATIF),L,X,None,None,Changement anormal,O,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
709,08800187243,ODQ,ODEUR (QUALITATIF),L,X,None,None,Non mesuré,O,NaN,2.0,0.0
710,08800187383,ODQ,ODEUR (QUALITATIF),T,X,None,None,Aucun changement anormal,O,1.0,0.0,NaN
711,08800187383,ODQ,ODEUR (QUALITATIF),L,X,None,None,Changement anormal,O,NaN,1.0,0.0
712,08800187454,ODQ,ODEUR (QUALITATIF),T,X,None,None,Aucun changement anormal,O,1.0,0.0,NaN


In [166]:
#Explo global resultats différent - types prélévements
query_type_referenceprel = """ 

SELECT *
FROM explo_global_resultat_diff
WHERE referenceprel IN ('00100143885','08800187383','01000107839','97300062104','03800341940' )
ORDER BY   qualitparam DESC, referenceprel, cdparametresiseeaux, insituana DESC 

"""
type_referenceprel = con.sql(query_type_referenceprel).df()
type_referenceprel

,referenceprel,cdparametresiseeaux,libmajparametre,insituana,cdunitereference,limitequal,refqual,rqana,qualitparam,valtraduite_1,valtraduite_0,valtraduite_2
0,01000107839,ODQ,ODEUR (QUALITATIF),T,X,<NA>,<NA>,Aucun changement anormal,O,2.0,0.0,NaN
1,01000107839,ODQ,ODEUR (QUALITATIF),L,X,<NA>,<NA>,Non mesuré,O,NaN,2.0,0.0
2,08800187383,ODQ,ODEUR (QUALITATIF),T,X,<NA>,<NA>,Aucun changement anormal,O,1.0,0.0,NaN
3,08800187383,ODQ,ODEUR (QUALITATIF),L,X,<NA>,<NA>,Changement anormal,O,NaN,1.0,0.0


#### Explo insituana ( T - L )

Les différences de résultats concernent les analyses faites selon :   
-  Terrain vs  Labo
-  Labo vs  Labo

104 prélévement, analysé seulement une fois sur Terrain    
3 prélévement, analysé seulement une fois en laboratoire  
90 prélévement, analysé en Laboratoire et une deuxième fois en laboratoire dont 12 présentent des différences de résultats  
16052 prélévement, analysé une première fois sur Terrain ensuite en Laboratoire dont 13910 présentent des différences de résultats  


In [167]:
#Explo global type d'analyse (Labo / terrain)

query_type_analyse = """ 

WITH type_insituana AS(
SELECT referenceprel, cdparametresiseeaux, valtraduite_1, valtraduite_0, valtraduite_2, 
LAG(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux, insituana DESC ) AS insituana_1, 
insituana AS insituana_0, 
LEAD(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux, insituana DESC ) AS insituana_2,
FROM resultat_plusieurs_analyses
ORDER BY referenceprel, cdparametresiseeaux)
,
combi_insituana AS (
SELECT
CASE 
WHEN insituana_1 IS NULL AND insituana_2 IS NOT NULL THEN insituana_0||insituana_2
WHEN insituana_1 IS NOT NULL AND insituana_2 IS NULL THEN insituana_1||insituana_0
ELSE insituana_0
END AS type_insituana
FROM type_insituana)

SELECT type_insituana, COUNT(*) AS nb
FROM combi_insituana
GROUP BY type_insituana

"""
type_analyse = con.sql(query_type_analyse).df()
type_analyse

,type_insituana,nb
0,TL,16052
1,T,104
2,LL,90
3,L,7


In [168]:
query_freq_insituana_valdiff = """ 
WITH type_insituana AS(
SELECT referenceprel, cdparametresiseeaux, valtraduite_1, valtraduite_0, valtraduite_2, 
LAG(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux, insituana DESC ) AS insituana_1, 
insituana AS insituana_0, 
LEAD(insituana ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux, insituana DESC ) AS insituana_2,
FROM explo_global_resultat_diff
ORDER BY referenceprel, cdparametresiseeaux)
,
combi_insituana AS (
SELECT
CASE 
WHEN insituana_1 IS NULL AND insituana_2 IS NOT NULL THEN insituana_0||insituana_2
WHEN insituana_1 IS NOT NULL AND insituana_2 IS NULL THEN insituana_1||insituana_0
ELSE insituana_0
END AS type_insituana
FROM type_insituana)

SELECT type_insituana, COUNT(*) AS nb
FROM combi_insituana
GROUP BY type_insituana

 """
freq_insituana_valdiff = con.sql(query_freq_insituana_valdiff).df()
freq_insituana_valdiff



,type_insituana,nb
0,TL,714


### Les paramètres concerné par plusieurs analyses

Sur l'ensemble des paramètres, 1,46 % font l'objet de plusieurs analyses (chiffre aléatoire, peut être largement plus voir 20%) dont 60 % présente des résultats différents)

In [169]:
query_paramtètre = """  
WITH 
paramètre_plusieurs_resultats AS (
SELECT DISTINCT cdparametresiseeaux
FROM plusieurs_analyses )
,
paramètre_global AS (
SELECT DISTINCT cdparametresiseeaux
FROM edc_resultats
)
,
resultat_diff AS (
SELECT DISTINCT cdparametresiseeaux
FROM explo_global_resultat_diff
)

SELECT COUNT(DISTINCT pps.cdparametresiseeaux) AS nbparam, COUNT (DISTINCT prg.cdparametresiseeaux) AS nbparamtotal, COUNT (DISTINCT rtd.cdparametresiseeaux) AS nbparamresultatdiff,
    ROUND (nbparam / nbparamtotal * 100 , 2) AS prctglob,ROUND (nbparamresultatdiff / nbparam * 100, 2) AS prctplsresultat
FROM paramètre_plusieurs_resultats AS pps
CROSS JOIN paramètre_global AS prg
CROSS JOIN resultat_diff AS rtd
"""
paramètre = con.sql(query_paramtètre).df()
paramètre



,nbparam,nbparamtotal,nbparamresultatdiff,prctglob,prctplsresultat
0,15,1049,1,1.43,6.67


## Conclusion ##

Un paramètre issu d'un même prélévement peut faire l'objet de plusieurs analyses dont une première analyse sur terrain et plusieurs analyses en laboratoires.  

Les analyses sur terrain (insituana = T ) ne sont pas référencé - pas de référence d'analyse 'referenceanl'  

Les résultats contradictoires sont présents sur des configuration :  
    - analyse 1 sur Terrain - analyse 2 en Laboratoire ( insituana T - L )  
    - analyse 1 en Laboratoire - analyse 2 en Laboratoire (ou plus) ( insituana L - L )  

Dans la table edc_resultats, 1,5 % des paramètres ont été analysés plus d'une fois dont 60 % présentents des résultats contradictoires  

Les informations de la table edc_resultats ne permet pas d'identifier les raisons exact qui amène à ces contradictions ( à creuser ?? ) :  
les lignes concernées présente des valeurs identiques pour chaque champ à part :  referenceanl - rqana - valtraduite  



